In [6]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [ ]:
display(melb_df.info())

In [ ]:
display(melb_df['Date'])

In [ ]:
#Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) 
# продажи объектов недвижимости. Найдите второй по популярности квартал продажи. 
# В качестве ответа запишите число объектов, проданных в этом квартале.
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
quarter_sold = melb_df['Date'].dt.quarter
print(quarter_sold.value_counts().iloc[1])


In [ ]:
# Преобразуйте все столбцы, в которых меньше 150 уникальных значений, 
# в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom,
# Bathroom, Car.
# В качестве ответа запишите результирующее количество столбцов, 
# которые имеют тип данных category.
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
max_unique_count = 150 
for col in melb_df.columns: 
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category')
display(melb_df.info())

### МЕТОД SORT_VALUES()
#### Основные параметры метода sort_values()

##### by — имя или список имён столбцов, по значениям которых производится сортировка.
##### axis — ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию сортировка производится по строкам.
##### ascending — сортировка по возрастанию (от меньшего к большему). По умолчанию параметр выставлен на True, для сортировки по убыванию (от большего к меньшему) необходимо выставить его на False.
##### ignore_index — создаются ли новые индексы в таблице. По умолчанию выставлен на False и сохраняет индексы изначальной таблицы.
##### inplace — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на False, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на True.

In [ ]:
# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):
melb_df.sort_values(by='Price').head(10)

In [ ]:
# А теперь отсортируем таблицу по убыванию 
# (от самой последней до самой первой) даты продажи объекта (Date).
# Для этого выставим параметр ascending на False:
melb_df.sort_values(by='Date', ascending=False)

### СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

#### Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка. При этом важно обращать внимание на порядок следования столбцов.

In [ ]:
# отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), 
# а затем — по возрастанию цены объекта (Price). 
# Для того чтобы вывод был более наглядным, выделим 
# каждую десятую строку из столбцов Distance и Price результирующей таблицы:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]
# Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. 
# Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым,
# то внутри такой группы производится сортировка по цене объекта.

In [ ]:
# Ради интереса попробуйте поменять порядок следования 
# столбцов в параметре by метода sort_values() и сравните результат. 
melb_df.sort_values(by=['Price', 'Distance']).loc[::10, ['Distance', 'Price']]

### КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ
##### Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, у которых площадь участка существенно больше площади здания, чтобы понять, как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

In [ ]:
# 2.Найдём информацию о таунхаусах (Type), 
# 3.проданных компанией (SellerG) McGrath, 
# 1.у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. 
# 4.Результат отсортируем по дате продажи (Date) в порядке возрастания, 
# 5.а после проведём сортировку по убыванию коэффициента соотношения площадей. 
# 6.Также обновим старые индексы на новые, установив параметр ignore_index на True. 
# 7.Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

###### https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
###### документация по методу sort_values()

In [ ]:
# Произведите сортировку столбца AreaRatio по убыванию. 
# При этом индексы полученной таблицы замените на новые. 
# Какое значение площади здания находится в строке 1558? 
# Ответ округлите до целого числа.
int(melb_df.sort_values(
    by='AreaRatio', 
    ignore_index=True,
    ascending=False
).loc[1558, 'BuildingArea'])

In [ ]:
# 1.Найдите таунхаусы (Type) 
# 2.с количеством жилых комнат (Rooms) больше 2. 
# 4.Отсортируйте полученную таблицу сначала по возрастанию числа комнат, 
# 5.а затем по убыванию средней площади комнат (MeanRoomsSquare). 
# 6.Индексы таблицы замените на новые. 
# 7.Какая цена будет у объекта в строке 18? 
# 3.Ответ запишите в виде целого числа.
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
int(melb_df[mask1&mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

## Группировка данных в DataFrame
### МЕТОД GROUPBY()
#### В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод groupby()
#### Основные параметры метода groupby()
#### by — имя или список имён столбцов, по которым производится группировка.
#### axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
#### as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.
#### Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 
#### Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

In [ ]:
# Применим агрегирующую функцию среднего к результату работы groupby(). 
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):
melb_df.groupby(by='Type').mean()
# Мы получили таблицу, на пересечении строк и
# столбцов которой находятся средние значения каждого числового признака в наших данных.

In [ ]:
# Как правило, нам не нужна информация обо всех столбцах, 
# поэтому агрегирующие методы можно применять только к интересующему нас столбцу. 
# Например, давайте сравним средние цены на объекты в зависимости от их типа:
melb_df.groupby('Type')['Price'].mean()
#Обратите внимание, что, так как мы считаем только один показатель (среднее) 
# для одного столбца, в результате мы получаем объект Series.

In [ ]:
# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
# Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости
# от его региона. Результат отсортируем по убыванию расстояния:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)
# наиболее удалёнными являются все регионы Victoria.


### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ
#### Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), который принимает список строк с названиями агрегаций.

#### Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания:

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)
# Обратите внимание, что, так как мы считаем несколько показателей для
# одного столбца, в результате мы получаем объект DataFrame.

In [ ]:
# Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, 
# вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':
melb_df.groupby('MonthSale')['Price'].agg('describe')

#### После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'.

In [ ]:
# Более того, метод agg() поддерживает использование и других функций. 
# Передадим дополнительно встроенную функцию set, чтобы получить множество из 
# агентств недвижимости, которые работают в каждом из регионов:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)
# Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria, 
# а наибольшая — в центральном районе Northern Metropolitan.

In [ ]:
#Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов
# недвижимости в каждой группе. В качестве ответа запишите количество комнат, 
# для которых средняя цена наибольшая.
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

In [ ]:
# Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
# Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
# В качестве ответа запишите название этого региона.
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
# Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период 
# с 1 мая по 1 сентября (включительно) 2017 года?
# Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
# Не забудьте перевести даты в формат datetime.
# добавил первую строку 
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

## Сводные таблицы
### МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ
#### На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода groupby — мы рассматривали сводную таблицу в контексте группировки по одному признаку.

In [ ]:
# Например, мы можем построить зависимость медианной цены и площади здания от числа комнат:
melb_df.groupby(['Rooms'])[['Price', 'BuildingArea']].median()

#### Но можно пойти дальше и построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

In [ ]:
# Чтобы финальный результат был представлен в виде таблицы,
# к результату агрегирования применим метод unstack():
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

#### В результате мы получаем сводную таблицу, столбцы в которой представляют типы домов (house, townhouse, unit), строки — число комнат, а на пересечении строк и столбцов находится средняя стоимость объекта с такими показателями.

### МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ
#### Основные параметры метода pivot_table()

##### values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
##### index — имя столбца, значения которого станут строками сводной таблицы;
##### columns — имя столбца, значения которого станут столбцами сводной таблицы;
##### aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
##### fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

#### Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение 0.

In [ ]:
# Финальный результат для наглядности вывода округлим с помощью метода round() до целых.
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

#### А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

#### Разберём ещё один пример. Найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

In [ ]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

## МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ
### Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр index или параметр columns.

#### Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

In [ ]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)


## ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ
#### Как получить доступ к данным или произвести фильтрацию в сложной сводной таблице, где есть дополнительные индексы?
#### Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.
#### Возьмём сводную таблицу, которую мы создавали ранее:

In [ ]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

In [ ]:
# Выведем её столбцы с помощью атрибута columns:
pivot.columns

#### Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, просто последовательно обратившись по имени столбцов:

In [ ]:
display(pivot['mean']['unit'])

#### Аналогично производится и фильтрация в данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

In [ ]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

#### Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:

In [ ]:
print(list(filtered_pivot.index))

#### Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая? В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

In [ ]:
pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(pivot)

#### Составьте сводную таблицу, которая показывает зависимость средней цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type). Во вновь созданной таблице найдите агентство, у которого средняя цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [ ]:
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='mean',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

## Объединение DataFrame: знакомимся с новыми данными
### С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?
#### В этой части модуля мы будем работать с популярным датасетом MovieLens, в котором собраны логи некоторой рекомендательной системы фильмов.

### Наши данные представляют собой четыре таблицы:
#### 1.2. ratings1 и ratings2 — части одной таблицы с данными выставленных пользователями оценок для фильмов.
##### - userId — уникальный идентификатор пользователя, который выставил оценку;
##### - movieId — уникальный идентификатор фильма;
##### - rating — рейтинг фильма.
#### 3. dates — таблица с датами выставления всех оценок.
##### - date — дата и время выставления оценки фильму.
#### 4. movies — таблица с информацией о фильмах.
##### - movieId — уникальный идентификатор фильма;
##### - title — название фильма и год его выхода;
##### - genres — жанры фильма.

### Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:
#### 1 Склеим таблицы ratings1 и ratings2 в единую структуру.
#### 2 К полученной таблице с рейтингами присоединим столбец с датой выставления рейтинга.
#### 3 Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

In [8]:
# Сколько уникальных фильмов представлено в таблице movies?
import pandas as pd
pd.__version__
movies_data = pd.read_csv('data/movies_data/movies.csv', sep=',')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies_df = movies_data.copy() # СОЗДАНИЕ КОПИИ ТАБЛИЦЫ ! ! !
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
print(movies_df['movieId'].nunique())
# 9742 уникальных фильмов представлено в таблице movies

9742


In [11]:
# Сколько уникальных пользователей в таблице ratings1?
ratings1_data = pd.read_csv('data/movies_data/ratings1.csv', sep=',')
ratings1_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [12]:
ratings1_df = ratings1_data.copy() # СОЗДАНИЕ КОПИИ ТАБЛИЦЫ ! ! !
ratings1_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [13]:
print(ratings1_df['userId'].nunique())
# 274 уникальных пользователей в таблице ratings1

274


In [14]:
# В каком году было выставлено больше всего оценок?
# Для ответа на этот вопрос используйте таблицу dates.
dates_data = pd.read_csv('data/movies_data/dates.csv', sep=',')
dates_data.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


In [15]:
dates_df = dates_data.copy() # СОЗДАНИЕ КОПИИ ТАБЛИЦЫ ! ! !
dates_df.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


In [16]:
pd.to_datetime(dates_df['date']).dt.year.value_counts
# В 2000 году было выставлено больше всего оценок

<bound method IndexOpsMixin.value_counts of 0         2000
1         2000
2         2000
3         2000
4         2000
          ... 
100831    2017
100832    2017
100833    2017
100834    2017
100835    2017
Name: date, Length: 100836, dtype: int64>

### Объединение DataFrame: concat
#### Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам. Для этого воспользуемся встроенной функцией Pandas concat(), которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

##### Основные параметры метода concat()

##### objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
##### axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
##### join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
##### ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.
###### Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

In [17]:
ratings2_data = pd.read_csv('data/movies_data/ratings2.csv', sep=',')
ratings2_data.head()

,userId,movieId,rating
0,274,5621,2.0
1,274,5630,3.0
2,274,5667,3.5
3,274,5679,3.5
4,274,5690,3.0


In [18]:
ratings2_df = ratings2_data.copy() # СОЗДАНИЕ КОПИИ ТАБЛИЦЫ ! ! !
ratings2_df.head()

,userId,movieId,rating
0,274,5621,2.0
1,274,5630,3.0
2,274,5667,3.5
3,274,5679,3.5
4,274,5690,3.0


In [19]:
# Итак, давайте склеим таблицы ratings1 и ratings2. 
# Для этого передадим их в списке в функцию concat(). 
# По умолчанию объединение происходит по строкам, 
# что нам и нужно, поэтому параметр axis мы не трогаем.
ratings = pd.concat([ratings1_df, ratings2_df])
display(ratings)
# В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.
# На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим
# на индексы таблицы, то заметим, что их нумерация не совпадает с количеством строк в 
# таблице, то есть у нас есть дублирующиеся индексы. Это может привести к некорректному
# объединению таблиц по ключевым столбцам.

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


In [20]:
# Это связано с тем, что по умолчанию concat сохраняет первоначальные 
# индексы объединяемых таблиц, а обе наши таблицы индексировались, 
# начиная от 0. Чтобы создать новые индексы, 
# нужно выставить параметр ignore_index на True:
ratings = pd.concat(
    [ratings1_df, ratings2_df],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [21]:
# Казалось бы, совсем другое дело! Но это ещё не всё. 
# Давайте узнаем количество строк в таблицах ratings и dates,
# ведь нам предстоит объединять их по столбцам:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates_df.shape[0])
print(ratings.shape[0] == dates_df.shape[0])
# Размерность таблиц разная — как такое могло произойти?


Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [22]:
# На самом деле очень просто: при выгрузке данных информация об оценках какого-то 
# пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации 
# случилось дублирование строк. В данном примере их легко найти — выведем последнюю 
# строку таблицы ratings1 и первую строку таблицы ratings2:
display(ratings1_df.tail(1))
display(ratings2_df.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


In [23]:
# Чтобы очистить таблицу от дублей, мы можем воспользоваться методом 
# DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. 
# Не забываем обновить индексы после удаления дублей, 
# выставив параметр ignore_index в методе drop_duplicates() на значение True:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [24]:
# Наконец, мы можем добавить к нашей таблице с оценками даты их выставления.
# Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings, dates_df], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


##### Задание 6.3 

In [ ]:
import pandas as pd
import os

def concat_users_files(path):
    """
    Вам необходимо написать функцию concat_user_files(path), параметром 
    которой является path - путь до директории. 
    Функция должна объединить информацию из предоставленных вам
    файлов в один DataFrame и вернуть его. 
    Не забудьте обновить индексы результирующей таблицы после объединения.
    Учтите тот момент, что в результате объединения могут 
    возникнуть дубликаты, от которых необходимо будет избавиться. 
    """
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data

if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

## Объединение DataFrame: join, merge
#### У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют объединением по ключевому столбцу.

#### Существует два основных типа объединения:
### inner (внутреннее)
##### При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.
#####  Аналогия в теории множеств - Пересечение (intersection) множеств А и В.
##### Строки, для которых совпадение не было найдено, удаляются.
### outer (внешнее)
#####  Данный тип делится на три подтипа:
#####  full — используется как outer по умолчанию, объединяет все варианты в обеих таблицах.
#####  Аналогия в теории множеств - Объединение (union) множеств А и В.
#####  left — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.
#####  Аналогия в теории множеств - Вычитание (difference) множества B из результата объединения (union) множеств А и В.
#####  right — аналогично предыдущему, но остаются значения только из «правой» таблицы. 
#####  Аналогия в теории множеств - Вычитание (difference) множества А из результата объединения (union) множеств А и В.
#####  Открыть примечание - Во всех трёх случаях, если совпадений между таблицами не найдено, на этом месте ставится пропуск (NaN).

## МЕТОД ОБЪЕДИНЕНИЯ JOIN

#### Для объединения двух таблиц по индексам используется метод DataFrame join(). Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).
##### Основные параметры метода join()
##### other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
##### how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
##### on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
##### lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

#### Если использовать метод join() «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.

In [25]:
# Проверим это, объединив таблицы типом left. Так как в наших таблицах есть 
# одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:
joined_false = ratings_dates.join(
    movies_df,
    rsuffix='_right',
    how='left'
)
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN,NaN,NaN


###### При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения left.

In [27]:
# Попробуйте изменить тип объединения, чтобы посмотреть на разницу
# результирующих таблиц.
joined_false = ratings_dates.join(
    movies_df,
    rsuffix='_left',
    how='right'
)
display(joined_false)

,userId,movieId,rating,date,movieId_left,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
9737,64,3481,4.0,2006-10-22 12:37:45,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,64,3489,3.0,2006-10-22 23:28:09,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,64,3499,4.5,2006-10-22 23:26:41,193585,Flint (2017),Drama
9740,64,3510,3.0,2006-10-22 23:27:26,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


#### Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right). Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. 

In [31]:
# Это можно сделать с помощью метода set_index().
# Также необходимо указать название ключа в параметре on.
joined = ratings_dates.join(
    movies_df.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


#### В результате такого объединения мы для каждого идентификатора фильма movieId в таблице ratings_dates нашли совпадающий с ним идентификатор movieId в таблице movies и присоединили информацию о самом фильме (title и genres). Это как раз то, что было нам нужно. Обратите внимание, что в результате такого объединения у нас остался лишь один столбец movieId.

## МЕТОД ОБЪЕДИНЕНИЯ MERGE
#### Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от join(), метод merge() предлагает более гибкий способ управления объединением, благодаря чему является более популярным.

### Основные параметры метода merge()

##### right — присоединяемая таблица. По умолчанию она является «правой».
##### how — параметр типа объединения. По умолчанию принимает значение 'inner'.
##### on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.
##### left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.
##### right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.
##### lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

### → Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не требует установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, данный метод позволяет объединять даже таблицы с разноимёнными ключами. Таким образом, merge() проще в использовании и более многофункционален, чем схожие методы.

In [36]:
# Посмотрим_ на метод merge() в действии.
# Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:
merged = ratings_dates.merge(
    movies_df,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [35]:
# Осталось только проверить, что число строк в 
# таблице ratings_dates совпадает с числом строк в результирующей таблице merged:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])
# Всё прошло успешно: для каждой оценки пользователя мы нашли информацию о фильме,
# которому она была выставлена.

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


## ОСОБЕННОСТИ ИСПОЛЬЗОВАНИЯ MERGE()
##### Возникает вопрос: почему мы выбрали тип объединения left, а не full, например?

In [37]:
# Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, 
# но с параметром how='outer' (full outer) и
# выведем размер таблицы, а также её «хвост»:
merged2 = ratings_dates.merge(
    movies_df,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
display(merged2.tail())
# Результирующее число строк в таблице увеличилось. Но за счёт чего?

Число строк в таблице merged2:  100854


,userId,movieId,rating,date,title,genres
100849,NaN,30892,NaN,NaN,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,NaN,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,NaN,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,NaN,"Browning Version, The (1951)",Drama
100853,NaN,85565,NaN,NaN,Chalet Girl (2011),Comedy|Romance


##### Оказывается в таблице movies содержались фильмы, которым ещё не были выставлены оценки. В результате объединения типом full outer мы дополнительно извлекли из таблицы movies информацию об этих фильмах, однако, так как оценки им ещё не были выставлены, мы получили пропуски (NaN) на местах столбцов таблицы ratings_dates (такие фильмы были записаны в конец таблицы). Важно! Учитывайте такие нюансы при работе с несколькими таблицами и проверяйте результат объединения.
##### → Метод merge() с внешним (outer) типом объединения может использоваться как аналог метода concat() при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.

In [38]:
# Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали раньше,
# но теперь используем метод merge():
merge_ratings = ratings1_df.merge(ratings2_df, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

Число строк в таблице merge_ratings:  100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


## Обратите внимание, что при использовании метода merge() для склейки двух таблиц у нас автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это особенность метода merge() — автоматическое удаление дублей.

In [42]:
# задание 7.5
import pandas as pd

items_df = pd.DataFrame({
'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
"""
Сформируйте DataFrame merged, в котором в результате объединения
purchase_df и items_df останутся модели, которые учтены на складе и имели продажи. 
"""
merged = items_df.merge(purchase_df, how='inner', on='item_id')
"""
Найдите из таблицы merged суммарную выручку, которую можно было бы получить 
от продажи всех товаров, которые есть на складе. 
Результат занесите в переменную income.
"""

income = (merged['price'] * merged['stock_count']).sum()
print(income)

19729490
